In [38]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data
import spotipy.util as util
import zipfile
import warnings
warnings.filterwarnings("ignore")

In [39]:
# Reading our data stored in a csv
audio_feats = pd.read_excel('../original/Hot 100 Audio Features.xlsx')
zf = zipfile.ZipFile('../notebooks/data/complete_100.csv.zip') 
complete = pd.read_csv(zf.open('complete_100.csv'),index_col=0)

In [41]:
# Keep only top 1
top = complete[complete['Week Position'] == 1]

# Merge top ten music with audio features to get a dataframe with the features of all the top 10 songs
audio_genre = top.merge(right=audio_feats, how='inner', on=['SongID'])[['WeekID', 'Week Position', 'Song_x', 'Performer_x', 'SongID', 'spotify_track_id']]

audio_genre['WeekID'] = (pd.to_datetime(audio_genre['WeekID']).dt.year)

In [42]:
# Select songs
song_to_keep = audio_genre[['WeekID','spotify_track_id','SongID']].groupby(['WeekID','spotify_track_id']).agg('count').sort_values(by='SongID').reset_index().drop_duplicates(['WeekID'],keep='last').spotify_track_id.values

In [43]:
# Dataframe from songs
df_song_playlist = audio_genre[audio_genre.spotify_track_id.isin(set(song_to_keep))].drop_duplicates('Song_x').sort_values(by='WeekID')

In [44]:
# Tracks of music of interest
tracks_id = df_song_playlist.spotify_track_id.to_list()

In [45]:
# Params of spotify creditential
client_id= "922e0977111543faae64a89c4cc49932"
client_secret= "6c7aea8528684421b1883ac051f10931"
username = 'khalfiyassine753'
scope = 'playlist-modify-public'
token = util.prompt_for_user_token(username,scope,client_id=client_id,client_secret=client_secret,redirect_uri='http://localhost:8008/callback') 
sp = spotipy.Spotify(auth=token)

In [46]:
# Creation of playlist
playlist_name = f"The most popular music since the 1950s from Billboard"    
sp.user_playlist_create(username,name=playlist_name)

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/2BrJrj9YAUnf4TA4ZYGUxu'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/2BrJrj9YAUnf4TA4ZYGUxu',
 'id': '2BrJrj9YAUnf4TA4ZYGUxu',
 'images': [],
 'name': 'The most popular music since the 1950s from Billboard',
 'owner': {'display_name': 'khalfiyassine753',
  'external_urls': {'spotify': 'https://open.spotify.com/user/khalfiyassine753'},
  'href': 'https://api.spotify.com/v1/users/khalfiyassine753',
  'id': 'khalfiyassine753',
  'type': 'user',
  'uri': 'spotify:user:khalfiyassine753'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSxlODFlMjIyNDE5MmMyODNiMzkxZWZhZGMzMDhkMmM5NDZmNzJiZGM4',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/2BrJrj9YAUnf4TA4ZYGUxu/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:2BrJ

In [47]:
# Get ID of playlist based on name
def GetPlaylistID(username, playlist_name):
    playlist_id = ''
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:  # iterate through playlists I follow
        if playlist['name'] == playlist_name:  # filter for newly created playlist
            playlist_id = playlist['id']
    print("Got Playlist ID.")
    return playlist_id

In [48]:
# Add songs to playlist
sp.user_playlist_add_tracks(username,GetPlaylistID(username,playlist_name), tracks_id)

Got Playlist ID.


{'snapshot_id': 'MixhMGExMTYzYjkyZTg5MTkxMWRiMGM2ZTBlNWQ4ODMyZWZkMjMyODdj'}